# Imports

In [1]:
!pip install clean-text
!pip install protobuf==4.25.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha256=9e7e05a4b6f692860ca1474731fb6c031d71108ec543273ba232abd8d22cd7ee
  Stored in directory: /root/.cache/pip/wheels/bd/22/e5/b69726d5e1a19795ecd3b3e7464b16c0f1d019aa94ff1c8578
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.15.0
    Uninstalling emoji-2.15.0:
      Successfully uninstalled emoji-2.15.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [2]:
# Pytorch related
import torch
import torch.nn as nn

# sklearn and scipy related
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from scipy import stats

# Huggingface related
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForMaskedLM, 
    DataCollatorForLanguageModeling, 
    TrainingArguments, 
    Trainer, 
    AutoConfig, 
    AutoModelForSequenceClassification,
    EarlyStoppingCallback
)
from huggingface_hub import snapshot_download

# text preprocessing
from cleantext import clean
import re

# miscellanious
import math
import pandas as pd
import numpy as np
import json
import random
from tqdm import tqdm
import os

2025-11-07 06:13:57.907829: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762496038.090146      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762496038.142710      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Reddit Domain Adaptation

## Model

In [3]:
model_checkpoint = "bert-base-cased"
model_path = snapshot_download(repo_id=model_checkpoint, allow_patterns=["pytorch_model.bin", "config.json", "vocab.txt"])
model_path

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

'/root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForMaskedLM.from_pretrained(model_path)

Some weights of the model checkpoint at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer

BertTokenizerFast(name_or_path='/root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e', vocab_size=28996, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [6]:
mask_token = tokenizer.mask_token
mask_token_id = tokenizer.mask_token_id
mask_token, mask_token_id

('[MASK]', 103)

In [7]:
inputs = tokenizer("I love [MASK] food", return_tensors="pt")
outputs = model(**inputs)
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
logits = outputs.logits[0, mask_token_index, :]
predicted_token_id = logits.argmax(dim=-1)
print(tokenizer.decode(predicted_token_id))

the


In [8]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

## Dataset

In [9]:
dsr = load_dataset("Fredithefish/Reddit-TIFU")
dsr

tifu_collection.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/619 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'meta'],
        num_rows: 619
    })
})

In [10]:
dsr['train']['meta']

Column([{'source': 'Reddit'}, {'source': 'Reddit'}, {'source': 'Reddit'}, {'source': 'Reddit'}, {'source': 'Reddit'}])

In [11]:
dsr['train'][0]

{'title': 'TIFU by raising the flag upside down on a military base and causing local farmers to think the base was in distress.',
 'text': 'First things first, I am not, and never have been, in the military.  This story concerns fifteen year old me when I was a Sea Cadet, which is kind of like Boy Scouts but directly connected to the US Navy. We\'d wear the (at the time) blue camouflage US Navy uniforms, with a giant gold sign on our shirts and hats that made clear we weren\'t actually in the Navy.  Like us being literal children wasn\'t a clue enough...\n\nIn the summer and school breaks, we could go to camps.  You could go on an abbreviated Naval Diver course.  You could go to Seabee camp and learn a bit about combat engineering and shoot airsoft guns at each other in training. You could go on a course to learn to defensive drive Humvees, which I used as training hours for my learner\'s permit. I can\'t express in words how bizarre and amazing this group was when I was a kid. \n\nTo 

In [12]:
dsr['train']['text']

Column(['First things first, I am not, and never have been, in the military.  This story concerns fifteen year old me when I was a Sea Cadet, which is kind of like Boy Scouts but directly connected to the US Navy. We\'d wear the (at the time) blue camouflage US Navy uniforms, with a giant gold sign on our shirts and hats that made clear we weren\'t actually in the Navy.  Like us being literal children wasn\'t a clue enough...\n\nIn the summer and school breaks, we could go to camps.  You could go on an abbreviated Naval Diver course.  You could go to Seabee camp and learn a bit about combat engineering and shoot airsoft guns at each other in training. You could go on a course to learn to defensive drive Humvees, which I used as training hours for my learner\'s permit. I can\'t express in words how bizarre and amazing this group was when I was a kid. \n\nTo do any of these though, you had to go through Sea Cadet basic training camp, which is held at an Air National Guard base in Marseil

## Preprocessing

In [13]:
def preprocessing(ds):
    """
    Only removing urls and hashtags from the text.
    Why not lowercasing - because bold words in social media means the person is trying to shout or more focus is given there.
    Why not punctuation - punctuations have can alter the meaning of a sentence
    Why not emojis or emoticons - Social media or online texts have so much usage of them.
    """
    texts = ' '.join(ds['train']['text'])
    
    cleaned = clean(texts, lower=False, no_urls=True, no_punct=False, replace_with_url='')
    cleaned = re.sub(r'#\w+', '', cleaned)
    cleaned = ' '.join(cleaned.split())

    split_words = 128
    splits = cleaned.split()
    chunks = [' '.join(splits[i:i+split_words]) for i in range(0, len(splits), split_words)]

    return chunks

corpus = preprocessing(dsr)
len(corpus)

1991

In [14]:
print(type(corpus), len(corpus))
print(corpus[:3])

<class 'list'> 1991
["First things first, I am not, and never have been, in the military. This story concerns fifteen year old me when I was a Sea Cadet, which is kind of like Boy Scouts but directly connected to the US Navy. We'd wear the (at the time) blue camouflage US Navy uniforms, with a giant gold sign on our shirts and hats that made clear we weren't actually in the Navy. Like us being literal children wasn't a clue enough... In the summer and school breaks, we could go to camps. You could go on an abbreviated Naval Diver course. You could go to Seabee camp and learn a bit about combat engineering and shoot airsoft guns at each other in training. You could go on a course to", "learn to defensive drive Humvees, which I used as training hours for my learner's permit. I can't express in words how bizarre and amazing this group was when I was a kid. To do any of these though, you had to go through Sea Cadet basic training camp, which is held at an Air National Guard base in Marseil

In [15]:
ds = Dataset.from_dict({'text' : corpus})
ds = ds.train_test_split(test_size=0.2, seed=42)
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1592
    })
    test: Dataset({
        features: ['text'],
        num_rows: 399
    })
})

In [16]:
for i in range(3):
    print(ds['train'][i])

{'text': 'have a rather large house and can sleep about ten people comfortably. My bedroom is on the main fooor adjacent to the kitchen and the entertainment space. This will be relevant shortly. I had been told for months on end that my guests would be arriving Thursday morning. I had hired a maid to clean the entire house top to bottom on Tuesday with the exception of my bedroom which I prefer to clean myself. Cut to Wedensday morning before work. I was wanting to blow off some steam before the extended family arrive and had quite a morning with my extensive collection. I own seven Real Dolls with costumes ranging from Zero Suit Samus to Queen Maeve to Rey Skywalker. I cosplay dress all my real dolls.'}
{'text': "that meant I was into him and wanted to invite him into my home and let him slide into that 🤢 TL;DR - A guy knocked on my door asking for directions and I stupidly gave him my number trying to provide a side point of contact knowing how hard it is to stay sober when all your

## Guided Masking

In [17]:
!wget https://saifmohammad.com/WebDocs/Lexicons/NRC-Emotion-Lexicon.zip
!unzip NRC-Emotion-Lexicon.zip

--2025-11-07 06:14:27--  https://saifmohammad.com/WebDocs/Lexicons/NRC-Emotion-Lexicon.zip
Resolving saifmohammad.com (saifmohammad.com)... 192.185.17.122
Connecting to saifmohammad.com (saifmohammad.com)|192.185.17.122|:443... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


connected.
HTTP request sent, awaiting response... 200 OK
Length: 25878449 (25M) [application/zip]
Saving to: ‘NRC-Emotion-Lexicon.zip’

NRC-Emotion-Lexicon 100%[===================>]  24.68M  27.3MB/s    in 0.9s    

2025-11-07 06:14:28 (27.3 MB/s) - ‘NRC-Emotion-Lexicon.zip’ saved [25878449/25878449]

Archive:  NRC-Emotion-Lexicon.zip
   creating: NRC-Emotion-Lexicon/
  inflating: NRC-Emotion-Lexicon/Paper-Practical-Ethical-Considerations-Lexicons.pdf  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._Paper-Practical-Ethical-Considerations-Lexicons.pdf  
  inflating: NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-ForVariousLanguages.txt  

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



  inflating: NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._NRC-Emotion-Lexicon-Wordlevel-v0.92.txt  
  inflating: NRC-Emotion-Lexicon/Paper1_NRC_Emotion_Lexicon.pdf  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._Paper1_NRC_Emotion_Lexicon.pdf  
  inflating: NRC-Emotion-Lexicon/README.txt  
  inflating: NRC-Emotion-Lexicon/Paper-Ethics-Sheet-Emotion-Recognition.pdf  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._Paper-Ethics-Sheet-Emotion-Recognition.pdf  
  inflating: NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Senselevel-v0.92.txt  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._NRC-Emotion-Lexicon-Senselevel-v0.92.txt  
   creating: NRC-Emotion-Lexicon/OneFilePerLanguage/
  inflating: NRC-Emotion-Lexicon/ListOfLanguages-For-Which-Lexicon-Availabale.txt  
  inflating: NRC-Emotion-Lexicon/Paper2_NRC_Emotion_Lexicon.pdf  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._Paper2_NRC_Emotion_Lexicon.pdf  
   creating: NRC-Emotion-Lexicon/OneFil

In [18]:
filepath = '/kaggle/working/NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'
nrc = (
    pd.read_csv(filepath,
                names=["word", "emotion", "association"],
                sep='\t',
                skiprows=0,
                keep_default_na=False)
    .query("association == 1")
)
nrc

,word,emotion,association
19,abacus,trust,1
23,abandon,fear,1
25,abandon,negative,1
27,abandon,sadness,1
30,abandoned,anger,1
...,...,...,...
141461,zest,anticipation,1
141464,zest,joy,1
141466,zest,positive,1
141469,zest,trust,1


In [19]:
nrc['emotion'].unique()

array(['trust', 'fear', 'negative', 'sadness', 'anger', 'surprise',
       'positive', 'disgust', 'joy', 'anticipation'], dtype=object)

In [20]:
lexicon = nrc['word'].unique()
lexicon = set(map(lambda x : x.lower(), lexicon))
len(lexicon)

6453

In [21]:
def guided_masking(ds, mask_size=0.2, mask_token='[MASK]'):
    """
    Mask 20% of words in the text that appear in lexicon.
    If <20% masked, fill remainder with random masking.
    """
    text = ds["text"]
    words = re.findall(r"\w+|\S", text) 
    n_words = len([w for w in words if re.match(r"\w+", w)])
    
    if n_words == 0:
        ds["masked_text"] = text
        return ds
    
    # Mask positions where word is in lexicon
    maskable_indices = [i for i, w in enumerate(words) if w.lower() in lexicon]
    n_to_mask = max(1, int(mask_size * n_words))
    
    masked_indices = set(random.sample(maskable_indices, min(len(maskable_indices), n_to_mask)))
    
    # If not enough guided words, do random masking
    if len(masked_indices) < n_to_mask:
        remaining = n_to_mask - len(masked_indices)
        all_word_indices = [i for i, w in enumerate(words) if re.match(r"\w+", w)]
        available = list(set(all_word_indices) - masked_indices)
        random_indices = random.sample(available, min(remaining, len(available)))
        masked_indices.update(random_indices)

    masked_words = [mask_token if i in masked_indices else w for i, w in enumerate(words)]
    
    # Apply masking
    masked_text = ""
    for i, w in enumerate(masked_words):
        if i > 0 and re.match(r"\w+|\[MASK\]", w) and re.match(r"\w+|\[MASK\]", masked_words[i - 1]):
            masked_text += " "
        masked_text += w
    
    ds["masked_text"] = masked_text
    return ds

In [22]:
ds = ds.map(guided_masking)
ds

Map:   0%|          | 0/1592 [00:00<?, ? examples/s]

Map:   0%|          | 0/399 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'masked_text'],
        num_rows: 1592
    })
    test: Dataset({
        features: ['text', 'masked_text'],
        num_rows: 399
    })
})

In [23]:
for i in range(3):
    print(ds['train'][i])

{'text': 'have a rather large house and can sleep about ten people comfortably. My bedroom is on the main fooor adjacent to the kitchen and the entertainment space. This will be relevant shortly. I had been told for months on end that my guests would be arriving Thursday morning. I had hired a maid to clean the entire house top to bottom on Tuesday with the exception of my bedroom which I prefer to clean myself. Cut to Wedensday morning before work. I was wanting to blow off some steam before the extended family arrive and had quite a morning with my extensive collection. I own seven Real Dolls with costumes ranging from Zero Suit Samus to Queen Maeve to Rey Skywalker. I cosplay dress all my real dolls.', 'masked_text': 'have a [MASK] large house [MASK] can sleep about ten people comfortably.My bedroom is on the [MASK] fooor adjacent to the kitchen and the [MASK] space.This will be [MASK] [MASK].I had been told for months on end that my guests [MASK] be [MASK] Thursday morning.I had hi

In [24]:
def tokenize_function(example):
    # Encode masked text as input, original text as label
    model_inputs = tokenizer(
        example["masked_text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

    labels = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs

In [25]:
tokenized_ds = ds.map(tokenize_function, batched=True, remove_columns=ds["train"].column_names)
tokenized_ds

Map:   0%|          | 0/1592 [00:00<?, ? examples/s]

Map:   0%|          | 0/399 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1592
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 399
    })
})

In [26]:
batch_size = 64 
model_name = model_checkpoint.split("/")[-1]
output_dir = f"{model_name}-domain-adapted"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    eval_strategy="epoch",
    learning_rate=5e-5,  
    weight_decay=0.01,
    num_train_epochs=6, 
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False, 
    fp16=True,  
    save_steps=1000,  
    logging_steps=len(tokenized_ds["train"]) // (batch_size * 2), 
    report_to="none", 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    # data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipykernel_19/3033905897.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,3.526800,3.232284
2,2.934800,2.975597
3,2.680700,2.812432
4,2.528700,2.733523
5,2.387900,2.704684
6,2.368600,2.688581


TrainOutput(global_step=150, training_loss=2.8713270950317384, metrics={'train_runtime': 141.4212, 'train_samples_per_second': 67.543, 'train_steps_per_second': 1.061, 'total_flos': 628521898328064.0, 'train_loss': 2.8713270950317384, 'epoch': 6.0})

In [27]:
eval_results = trainer.evaluate()
perplexity = math.exp(eval_results["eval_loss"])
print(f"Perplexity: {perplexity:.2f}")

trainer.save_model() 

Perplexity: 14.71


## Testing

In [28]:
model_path = "/kaggle/working/bert-base-cased-domain-adapted" 
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForMaskedLM.from_pretrained(model_path)
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [29]:
def predict_masked_tokens(text, top_k=5):
    """
    Predict top-k tokens for each [MASK] in the text.
    """
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits

    masked_indices = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

    for idx in masked_indices:
        probs = predictions[0, idx].softmax(dim=0)
        topk = torch.topk(probs, k=top_k)
        tokens = tokenizer.convert_ids_to_tokens(topk.indices)
        scores = topk.values.tolist()

        print(f"\nMask position {idx.item()}:")
        for token, score in zip(tokens, scores):
            print(f"  {token:15s} | {score:.4f}")


In [30]:
text = "I have been feeling very [MASK] lately."
predict_masked_tokens(text)


Mask position 6:
  bad             | 0.1492
  sick            | 0.0808
  ill             | 0.0751
  uncomfortable   | 0.0535
  good            | 0.0307


In [31]:
text = "I find it hard to [MASK] in activities I used to enjoy."
predict_masked_tokens(text)


Mask position 6:
  participate     | 0.6518
  engage          | 0.2243
  believe         | 0.0318
  take            | 0.0115
  interest        | 0.0091


In [32]:
base_model = AutoModelForMaskedLM.from_pretrained("bert-base-cased")

def compare_models(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        base_logits = base_model(**inputs).logits
        domain_logits = model(**inputs).logits

    idx = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1][0]
    base_probs = base_logits[0, idx].softmax(dim=0)
    domain_probs = domain_logits[0, idx].softmax(dim=0)

    print("\nBase BERT predictions:")
    for token, score in zip(*zip(*[(t, s) for t, s in zip(
        tokenizer.convert_ids_to_tokens(torch.topk(base_probs, 5).indices),
        torch.topk(base_probs, 5).values.tolist()
    )])):
        print(f"  {token:15s} | {score:.4f}")

    print("\nDomain-adapted predictions:")
    for token, score in zip(*zip(*[(t, s) for t, s in zip(
        tokenizer.convert_ids_to_tokens(torch.topk(domain_probs, 5).indices),
        torch.topk(domain_probs, 5).values.tolist()
    )])):
        print(f"  {token:15s} | {score:.4f}")


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
original = "I used to be able to cry"
text = "I used to be able to [MASK]."
compare_models(text)


Base BERT predictions:
  fly             | 0.0607
  walk            | 0.0550
  talk            | 0.0547
  sleep           | 0.0492
  read            | 0.0480

Domain-adapted predictions:
  read            | 0.1089
  talk            | 0.0715
  breathe         | 0.0271
  communicate     | 0.0260
  laugh           | 0.0246


In [34]:
original = "I hate myself"
text = "I [MASK] myself"
compare_models(text)


Base BERT predictions:
  asked           | 0.2377
  ask             | 0.1121
  hate            | 0.0649
  told            | 0.0528
  tell            | 0.0355

Domain-adapted predictions:
  reminded        | 0.0808
  told            | 0.0652
  found           | 0.0521
  i               | 0.0313
  hate            | 0.0313


In [35]:
original = "I have thoughts of killing myself."
text = "I have thoughts of [MASK] myself."
compare_models(text)


Base BERT predictions:
  killing         | 0.0679
  it              | 0.0565
  you             | 0.0317
  that            | 0.0314
  him             | 0.0260

Domain-adapted predictions:
  killing         | 0.2434
  dying           | 0.0292
  being           | 0.0267
  me              | 0.0238
  death           | 0.0236


# Mental Domain Adaptation

In [36]:
dsm1 = load_dataset("ShreyaR/DepressionDetection")
dsm1

depression_dataset_reddit_twitter.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/7731 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'is_depression'],
        num_rows: 7731
    })
})

In [37]:
dsm2 = load_dataset("hugginglearners/reddit-depression-cleaned")
dsm2

README.md: 0.00B [00:00, ?B/s]

depression_dataset_reddit_cleaned.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/7731 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'is_depression'],
        num_rows: 7731
    })
})

In [38]:
dsm3 = load_dataset("jsfactory/mental_health_reddit_posts")
dsm3

posts.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/24000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['body', 'subreddit'],
        num_rows: 24000
    })
})

In [39]:
dsm3['train'][0]

{'body': 'Thank you.\n\nMy son was recently diagnosed and I want to do whatever I can to support and help him. His current teacher also has ADHD and has been wonderful. I’m hopeful for my son.',
 'subreddit': 1}

In [40]:
merged_dsm = (
    list(dsm1["train"]["clean_text"]) +
    list(dsm2["train"]["clean_text"]) +
    list(dsm3["train"]["body"])
)

print(f"Total merged samples: {len(merged_dsm)}")

Total merged samples: 39462


In [41]:
def preprocessing(merged_dsm):
    """
    Same as above, just the input types is changed.
    """
    texts = merged_dsm
    
    cleaned = clean(texts, lower=False, no_urls=True, no_punct=False, replace_with_url='')
    cleaned = re.sub(r'#\w+', '', cleaned)
    cleaned = ' '.join(cleaned.split())

    split_words = 128
    splits = cleaned.split()
    chunks = [' '.join(splits[i:i+split_words]) for i in range(0, len(splits), split_words)]

    return chunks

corpus = preprocessing(merged_dsm)
len(corpus)

17991

In [42]:
ds = Dataset.from_dict({'text' : corpus})
ds = ds.train_test_split(test_size=0.2, seed=42)
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 14392
    })
    test: Dataset({
        features: ['text'],
        num_rows: 3599
    })
})

In [43]:
ds = ds.map(guided_masking)
ds

Map:   0%|          | 0/14392 [00:00<?, ? examples/s]

Map:   0%|          | 0/3599 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'masked_text'],
        num_rows: 14392
    })
    test: Dataset({
        features: ['text', 'masked_text'],
        num_rows: 3599
    })
})

In [44]:
tokenized_ds = ds.map(tokenize_function, batched=True, remove_columns=ds["train"].column_names)
tokenized_ds

Map:   0%|          | 0/14392 [00:00<?, ? examples/s]

Map:   0%|          | 0/3599 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14392
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3599
    })
})

In [45]:
model_path = "/kaggle/working/bert-base-cased-domain-adapted"  
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForMaskedLM.from_pretrained(model_path)
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [46]:
batch_size = 64 
model_name = model_checkpoint.split("/")[-1]
output_dir = f"{model_name}-domain-adapted-mental-dataset"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    eval_strategy="epoch",
    learning_rate=5e-5,  
    weight_decay=0.01,
    num_train_epochs=6, 
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False, 
    fp16=True,  
    save_steps=1000,  
    logging_steps=len(tokenized_ds["train"]) // (batch_size * 2), 
    report_to="none", 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    # data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipykernel_19/4036430674.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,2.356400,2.239802
2,2.169700,2.149687
3,2.057000,2.114397
4,1.919200,2.152236
5,1.794300,2.299985
6,1.729800,2.341162


TrainOutput(global_step=1350, training_loss=2.025484944802743, metrics={'train_runtime': 1261.3168, 'train_samples_per_second': 68.462, 'train_steps_per_second': 1.07, 'total_flos': 5681964296945664.0, 'train_loss': 2.025484944802743, 'epoch': 6.0})

In [47]:
eval_results = trainer.evaluate()
perplexity = math.exp(eval_results["eval_loss"])
print(f"Perplexity: {perplexity:.2f}")

trainer.save_model() 

Perplexity: 10.39


In [48]:
def compare_three_models(text, tokenizer, base_model, domain_model1, domain_model2):
    """
    Compare predictions of base BERT and two domain-adapted models on a masked text.
    """
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        base_logits = base_model(**inputs).logits
        domain1_logits = domain_model1(**inputs).logits
        domain2_logits = domain_model2(**inputs).logits

    # Locate [MASK] token index
    idx = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1][0]

    # Convert logits to probabilities
    base_probs = base_logits[0, idx].softmax(dim=0)
    domain1_probs = domain1_logits[0, idx].softmax(dim=0)
    domain2_probs = domain2_logits[0, idx].softmax(dim=0)

    def print_top(pred_probs, label):
        print(f"\n{label} predictions:")
        top = torch.topk(pred_probs, 5)
        for token_id, score in zip(top.indices, top.values):
            token = tokenizer.convert_ids_to_tokens(token_id.item())
            print(f"  {token:15s} | {score:.4f}")

    # Show results
    print("="*60)
    print(f"Input: {text}")
    print("="*60)

    print_top(base_probs, "Base BERT")
    print_top(domain1_probs, "Domain Adapted 1")
    print_top(domain2_probs, "Domain Adapted 2")

    print("="*60)

In [49]:
base_model = AutoModelForMaskedLM.from_pretrained("bert-base-cased")
domain_adapted1 = AutoModelForMaskedLM.from_pretrained('/kaggle/working/bert-base-cased-domain-adapted')
domain_adapted2 = AutoModelForMaskedLM.from_pretrained("/kaggle/working/bert-base-cased-domain-adapted-mental-dataset")
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/bert-base-cased-domain-adapted")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [50]:
texts = [
    "I used to be able to [MASK].",
    "I [MASK] myself.",
    "I have thoughts of [MASK] myself."
]

for text in texts:
    compare_three_models(text, tokenizer, base_model, domain_adapted1, domain_adapted2)

Input: I used to be able to [MASK].

Base BERT predictions:
  fly             | 0.0607
  walk            | 0.0550
  talk            | 0.0547
  sleep           | 0.0492
  read            | 0.0480

Domain Adapted 1 predictions:
  read            | 0.1089
  talk            | 0.0715
  breathe         | 0.0271
  communicate     | 0.0260
  laugh           | 0.0246

Domain Adapted 2 predictions:
  focus           | 0.2003
  talk            | 0.0637
  communicate     | 0.0413
  eat             | 0.0342
  heal            | 0.0315
Input: I [MASK] myself.

Base BERT predictions:
  asked           | 0.3741
  ask             | 0.1328
  told            | 0.0488
  hate            | 0.0370
  tell            | 0.0237

Domain Adapted 1 predictions:
  hate            | 0.1065
  told            | 0.0566
  hated           | 0.0518
  reminded        | 0.0405
  asked           | 0.0351

Domain Adapted 2 predictions:
  hate            | 0.8118
  love            | 0.0249
  kill            | 0.0192
  blame     

# Fine Tuning

In [51]:
ground_truth_df = pd.read_csv('/kaggle/input/erisk25/final-eriskt2-dataset-with-ground-truth/final-eriskt2-dataset-with-ground-truth/shuffled_ground_truth_labels.txt', sep=' ', header=None)
ground_truth_df.columns = ['user', 'is_depressed']
ground_truth_df.head()

,user,is_depressed
0,subject_BAPb4sn,0
1,subject_x12ArlU,0
2,subject_D1lB5nr,0
3,subject_aEK9WJt,0
4,subject_OpiNZAB,0


In [52]:
posts = {}
root_dir = '/kaggle/input/erisk25/final-eriskt2-dataset-with-ground-truth/final-eriskt2-dataset-with-ground-truth/all_combined'
for file in os.listdir(root_dir):
    file_path = os.path.join(root_dir, file)

    # user id
    user_id = file.split('.')[0]

    # reading the json file
    with open(file_path, 'r') as f:
        file_data = json.load(f)

    user_posts = []
    # using the file data
    for post in file_data:
        if post['submission']['target'] == True:
            user_posts.append(post['submission']['body'])

        comments = post['comments']
        for comment in comments:
            if comment['target'] == True:
                user_posts.append(comment['body'])

    posts[user_id] = user_posts

In [53]:
print(len(posts.keys()))

909


In [54]:
def preprocess(user_posts, N=35):
    # filter removed posts
    user_posts = list(filter(lambda x : x != '[removed]', user_posts))
    texts = ' '.join(user_posts)

    # lowercasing + removing punctuations and urls 
    cleaned = clean(texts, lower=False, no_urls=True, no_punct=False, replace_with_url='')
    
    # removing hashtags
    cleaned = re.sub(r'#\w+', '', cleaned)
    cleaned = ' '.join(cleaned.split())

    if len(cleaned) == 0:
        return []

    # convert into 35 segments
    segment_size = math.ceil(len(cleaned) / N)
    segments = [cleaned[i : i+segment_size] for i in range(0, len(cleaned), segment_size)]

    if len(segments) < N:
        segments += [""] * (N - len(segments))
    elif len(segments) > N:
        segments = segments[:N]

    return segments

In [55]:
processed_posts = {}
for user_id, user_post in posts.items():
    processed_posts[user_id] = preprocess(user_post)

len(processed_posts)

909

In [56]:
for user_id in processed_posts.keys():
    print(processed_posts[user_id])
    print(len(processed_posts[user_id]))
    break

["I'll start watching it! Curious to see what they got Heya! So I've been an Elemental Hero fan since I can remember. Own the same deck as on Master Duel. Now that I've completed it, I would like to venture into so", "me new fun themes and possibly find a new favorite deck. I've seen Dinomorphia and really like the design of the cards, but it seems like they don't got a lot going on! I have not watched the anime after GX but d", "id play a bit with tuners. Is there any anime I should watch that has really fun and cool cards and mechanic introduced? The decks I'm currently working on are Chaos, cyber dark and crystal beast. None of them ha", "ve the same spark as E-hero though) Any recommendations?? Thanks :3 I've never seen these before 🤣 very cute! I'll look into it! I love the purple haired character in Unchained!! Definitely gonna look into this, ", "thanks for the recommendation! Also swordsoul's chinese theme is fire! Might make this my synchro deck 🔥 I saw the art from Phantom Kn

In [57]:
train_ids, test_ids = train_test_split(ground_truth_df, test_size=0.1, stratify=ground_truth_df['is_depressed'], random_state=42)


train_users = train_ids['user'].values
test_users  = test_ids['user'].values

len(train_users), len(test_users)

(818, 91)

In [58]:
train_labels = ground_truth_df[ground_truth_df['user'].isin(train_users)]['is_depressed']
print(train_labels.value_counts())

test_labels = ground_truth_df[~ground_truth_df['user'].isin(train_users)]['is_depressed']
print(test_labels.value_counts())

is_depressed
0    726
1     92
Name: count, dtype: int64
is_depressed
0    81
1    10
Name: count, dtype: int64


In [59]:
def build_examples(selected_users):
    texts, labels, users = [], [], []
    for uid in selected_users:
        segments = processed_posts[uid]
        label = ground_truth_df[ground_truth_df['user'] == uid]['is_depressed'].values[0]
        for seg in segments:
            texts.append(seg)
            labels.append(label)
            users.append(uid)
    return texts, labels, users

In [60]:
train_texts, train_labels, train_user = build_examples(train_users)

len(train_texts), len(train_labels), len(train_user)

(28070, 28070, 28070)

In [61]:
train_texts[0], train_labels[0], train_user[0]

('Mozda najbolje da se smuvas sa tom ribom. Sve dok je ljubav iskrena, ja podrzavam. Moras da imas jedno gay iskustvo da bi znao da nisi gay. Svi smo kroz to prosli, opusteno. You wouldnt stone good ol Dave, would you now? >da li imam pravo na neku zalbu ili ne Ne. >jer su dve razlicite stete u pitanju, i da sam uspeo da sredim auto nakon prve stete, svakako bi u drugoj morali da mi isplate isti iznos , jer je trebao da de menja ceo branik... Ne. U ovom slucaju nema nikakvog "natezanja" i sve je cisto. Batici je ostecen 100% ispravan branik, dali su mu 14 soma da to prefarba. On je odlucio da je bolje 14 soma u dzepu i ogrebotina nego farbanje i 0 soma u dzepu, nije popravio branik. Kada mu se desila sledeca stvar, na istom braniku, da li je taj branik sada opet bio 100% ispravan? Pa nije imao je stetu, ogrebotinu. Koliko vredi ta ogrebotina? Onoliko koliko je placeno da se sanira, 14k, toliko su mu odbili do novog branika, jer je to vec dobio. Sve je vrlo cisto i po zakonu. To sto ti 

In [62]:
test_texts, test_labels, test_user = build_examples(test_users)
len(test_texts), len(test_labels), len(test_user)

(3115, 3115, 3115)

In [63]:
test_texts[0], test_labels[0], test_user[0]

("Itachi uchiha not that I am a huge fan but man his story hits hard 1. Naruto Shipudden / OG Naruto 2. Attack on Titan 3. Saiki 4. Berserk (manga) 5.Demon Slayer / jujutsu kaisen Hashirama hit pics I have watches 6 of them including Naruto both parts but I am in cbse Dead space 2 when I was a kid Very bad tier Come on guys you all know it's Itachi uchiha One piece and Naruto fans be hating you Real life oututsuki Go to a country where dental care is cheap like India and get your teeth fixed it will cost you less than usual high prices I can fix my teeth under 400 dollors Bro they all contain same amount of DNA so even any one of them have been fused you will be still you but XX or XY sperm have been fused you would have a different gender Naruto-kun Well guys we don't lose wars like america Mob psycho 100 is shit and boring Bro told the truth and don't lie we all know that ep 1071 was at peak hype and the hype died at 1072 ep WOW loyal fans Your Twitter was attacked by T-ELON missile 

In [64]:
train_dataset = Dataset.from_dict({
    "text": train_texts,
    "label": train_labels,
    "user": train_user
})
test_dataset = Dataset.from_dict({
    "text": test_texts,
    "label": test_labels,
    "user": test_user
})

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'user'],
        num_rows: 28070
    })
    test: Dataset({
        features: ['text', 'label', 'user'],
        num_rows: 3115
    })
})

In [65]:
split = dataset["train"].train_test_split(test_size=0.1, seed=42)

dataset = DatasetDict({
    "train": split["train"],
    "val": split["test"],
    "test": dataset["test"]
})

dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'user'],
        num_rows: 25263
    })
    val: Dataset({
        features: ['text', 'label', 'user'],
        num_rows: 2807
    })
    test: Dataset({
        features: ['text', 'label', 'user'],
        num_rows: 3115
    })
})

In [66]:
model_path = "/kaggle/working/bert-base-cased-domain-adapted-mental-dataset"

config = AutoConfig.from_pretrained(model_path)
config.num_labels = 2

model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    config=config,
)
print(model.classifier)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/bert-base-cased-domain-adapted-mental-dataset and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Linear(in_features=768, out_features=2, bias=True)


In [67]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [68]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [69]:
tokenizer

BertTokenizerFast(name_or_path='/kaggle/working/bert-base-cased-domain-adapted-mental-dataset', vocab_size=28996, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [70]:
def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

tokenized_dataset = dataset.map(tokenize_fn, batched=True)
tokenized_dataset

Map:   0%|          | 0/25263 [00:00<?, ? examples/s]

Map:   0%|          | 0/2807 [00:00<?, ? examples/s]

Map:   0%|          | 0/3115 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'user', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25263
    })
    val: Dataset({
        features: ['text', 'label', 'user', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2807
    })
    test: Dataset({
        features: ['text', 'label', 'user', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3115
    })
})

In [71]:
train_labels = tokenized_dataset["train"]["label"]

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = torch.tensor(class_weights, dtype=torch.float)
print("Class weights:", class_weights)

Class weights: tensor([0.5647, 4.3632])


In [72]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds, pos_label=1),
        'recall': recall_score(labels, preds, pos_label=1), 
        'f1': f1_score(labels, preds, pos_label=1)
    }

In [73]:
class WeightedTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights.to(logits.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [74]:
args = TrainingArguments(
    output_dir="./bert-base-cased-finetuned",
    overwrite_output_dir=True,
    save_total_limit=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=100,
    weight_decay=0.01,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
    logging_dir="./logs",
)

trainer = WeightedTrainer(
    class_weights=class_weights,
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.463100,0.464499,0.745992,0.295789,0.864615,0.440784
2,0.334100,0.429632,0.889918,0.517391,0.732308,0.606369
3,0.240900,0.864423,0.918418,0.665517,0.593846,0.627642
4,0.149500,1.142287,0.917706,0.666667,0.578462,0.619440
5,0.090600,1.001262,0.905949,0.590504,0.612308,0.601208
6,0.049900,1.379915,0.910581,0.610778,0.627692,0.619120
7,0.049500,1.473178,0.916637,0.644444,0.624615,0.634375
8,0.025500,2.049771,0.921268,0.722222,0.520000,0.604651
9,0.029900,1.825383,0.912362,0.645756,0.538462,0.587248
10,0.035200,1.499393,0.911649,0.615616,0.630769,0.623100


TrainOutput(global_step=13430, training_loss=0.0950272744747568, metrics={'train_runtime': 5049.7764, 'train_samples_per_second': 500.28, 'train_steps_per_second': 15.644, 'total_flos': 2.824964201412864e+16, 'train_loss': 0.0950272744747568, 'epoch': 17.0})

## Testing

In [75]:
predictions = trainer.predict(tokenized_dataset["test"])

segment_preds = np.argmax(predictions.predictions, axis=1)
segment_labels = predictions.label_ids
user_ids = tokenized_dataset["test"]["user"]

df = pd.DataFrame({
    "user_id": user_ids,
    "segment_pred": segment_preds,
    "segment_label": segment_labels
})

def majority_vote(group):
    mode_result = stats.mode(group["segment_pred"], keepdims=True)
    return mode_result.mode[0]

user_preds_series = df.groupby("user_id").apply(majority_vote)
user_preds = user_preds_series.values
user_ids_unique = user_preds_series.index

user_labels_series = df.groupby("user_id")["segment_label"].first()
user_labels = user_labels_series.values

assert (user_preds_series.index == user_labels_series.index).all(), "User ID mismatch!"

accuracy = accuracy_score(user_labels, user_preds)
precision = precision_score(user_labels, user_preds, zero_division=0) 
recall = recall_score(user_labels, user_preds, zero_division=0)
f1 = f1_score(user_labels, user_preds, zero_division=0)

print(f"User-level Results:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

results_df = pd.DataFrame({
    "user_id": user_ids_unique,
    "true_label": user_labels,
    "pred_label": user_preds
})
results_df.to_csv("user_level_results.csv", index=False)

User-level Results:
Accuracy:  0.9663
Precision: 1.0000
Recall:    0.7000
F1 Score:  0.8235


/tmp/ipykernel_19/1365410557.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  user_preds_series = df.groupby("user_id").apply(majority_vote)
